In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
!ls CDATA

AUS.csv  FIN.csv  FRA.csv  GBR.csv


In [3]:
ccodes = ['AUS']#, 'FRA', 'GBR']
datas = ['./CDATA/{}.csv'.format(x) for x in ccodes]

In [4]:
datas[0]

'./CDATA/AUS.csv'

In [5]:
datas

['./CDATA/AUS.csv']

In [6]:
datas = [pd.read_csv(x).values.astype(floatX).T[np.newaxis][:,1:-1]/100 for x in datas]

In [7]:
[data.shape for data in datas]

[(1, 36, 3)]

In [8]:
# datas

In [9]:
scaler = 0.
for i, data in enumerate(datas):
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    scaler = scaler + stds
    datas[i] = data
print('---')
scaler /= len(datas)
for i in range(len(datas)):
    datas[i] /= scaler
    data = datas[i]
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
    datas[i] = data

[0.01887933 0.01939655 0.00931648]
---
[1. 1. 1.]


In [10]:
datas[0][:,:4,:3]

array([[[ 5.36384494, -0.01548735,  6.54753965],
        [ 5.13352048,  1.08681084,  6.22552981],
        [ 5.9035507 ,  0.72401918,  7.72824344],
        [ 5.35694906,  1.05816547, 10.73367173]]])

In [11]:
country_data = {c:d for c,d in zip(ccodes, datas)}

In [12]:
class VARmodel:
    def __init__(self, data, name='VARmodel', mu=None):
        self.data = data
        self.NUM_STEPS = data.shape[1]
        self.name = name
        self.logdensities = []
        self.priors = []
        self.dim = [3,3*2+1]
        self.summaries = []
        
        self.stop_time = tf.placeholder_with_default(self.NUM_STEPS, ())
        self.observable_mask = tf.range(0, data.shape[1], dtype=tf.int32) < self.stop_time
        
        pd = np.mean(np.std(data[0,1:] - data[0,:-1], axis=0))
#         print(pd)
        
        with tf.variable_scope(name) as scope:
            self.scope = scope
            
            self.create_rw_priors()
            self.outputs = self.create_walk_inference(mu=mu)
            self.create_observ_dispersion_inference(pd*0.5)
            self.create_likelihood(self.observable_mask, self.outputs)
            self.summary = tf.summary.merge(self.summaries)
            
    def create_summary(self, stype, name, tensor):
        s = stype(name, tensor)
        self.summaries.append(s)

    def create_rw_priors(self):
        dim = self.dim
        with tf.variable_scope('rw_priors'):
            with tf.variable_scope('walk_ord'):
                s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=6., name='s1_prior')

                with tf.variable_scope('s1_inference', dtype=floatX):
                    mu = tf.get_variable('mu', shape=[1], 
                                         initializer=tf.constant_initializer(s1_prior_d.mu))

                    logsigma_init = tf.constant_initializer(min(math.log(s1_prior_d.sigma), -1.))
                    logsigma = tf.get_variable('logsigma', shape=[1], 
                                               initializer=logsigma_init)
                    sigma = tf.exp(logsigma)
                    s1_d = LogNormal(1, mu=mu, sigma=sigma)

                s1 = s1_d.sample()
                
                self.create_summary(tf.summary.scalar, 's1_ord', s1[0])
                
                self.logdensities.append(s1_d.logdens(s1))

                s1_prior = s1_prior_d.logdens(s1)
                self.priors.append(s1_prior)
                
            PWalk = NormalRW(dim=None, sigma0=.4, mu0=0., sigma=s1, name='OrdWalk')
            print(PWalk.init_distr.sigma, PWalk.init_distr.mu)
            self.PWalk = PWalk
                
    def create_walk_inference(self, mu=None):
        dim = self.dim
        gvar = GVAR(dim=dim[0]*dim[1], len=self.NUM_STEPS, name='coef_rw_inference', mu=mu)
        outputs = gvar.sample()
        
        self.logdensities.append(gvar.logdens)
        
        print(self.PWalk.init_distr.sigma, self.PWalk.init_distr.mu)

        self.priors.append(self.PWalk.logdens(outputs, reduce=True))
        self.outputs = outputs
        
        return outputs
    
    def create_observ_dispersion_inference(self, prior_disp):
        with tf.variable_scope('obs_d_inf', reuse=tf.AUTO_REUSE):
#             ldiag = DFlow([NVPFlow(dim=3, name='ldiag_flow_' + str(i)) for i in range(2)], init_sigma=0.05)
            ldiag = DFlow([LinearChol(dim=3, name='ldiag_flow_' + str(i)) for i in range(1)], init_sigma=0.05)

            ldiag.output -= 0.5*math.log(prior_disp)
            ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)
#             print(ldiag.logdens)
#             print(ldiag.output)
        
        self.obs_d = MVNormal(3, sigma=None, name='obs_d_posterior', 
                   ldiag=ldiag.output[0])

#         self.obs_d = MVNormal(3, sigma=0.4, name='obs_d_posterior', 
#                    ldiag=None)
        
        df = 3
        pmat = np.diag([(2./prior_disp)]*3)/df
        cov_prior = WishartCholesky(df, pmat, cholesky_input_output_matrices=True)
        
        pr = cov_prior.log_prob(self.obs_d.fsigma)
        self.logdensities.append(ldiag.logdens[0])
        self.priors.append(pr)
        sigmas = tf.sqrt(tf.diag_part(self.obs_d.sigma))
        rsquareds = 1 - sigmas/np.std(self.data[0,1:,:3] - self.data[0,:-1,:3], axis=0)
        self.create_summary(tf.summary.scalar, 'rsquared_post_mean', tf.reduce_mean(rsquareds))
        self.create_summary(tf.summary.histogram, 'post_rsquared', rsquareds)
        self.create_summary(tf.summary.histogram, 'post_disp', tf.sqrt(tf.diag_part(self.obs_d.sigma)))

#         obs_d_prior = LogNormal(dim=None, mu=math.log(prior_disp), sigma=0.5)

#         with tf.variable_scope('obs_cov_inference', dtype=floatX):
#             mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(obs_d_prior.mu))
#             logsigma = tf.get_variable('logsigma', shape=[1], initializer=tf.constant_initializer(-5))
#             sigma = tf.exp(logsigma)

#             obs_d = LogNormal(1, mu=mu, sigma=sigma)
#             obs_ds = obs_d.sample()
#             self.obs_ds = obs_ds

#             obs_ds_logdens = obs_d.logdens(obs_ds)
            
#             self.logdensities.append(obs_ds_logdens)
#             self.priors.append(obs_d_prior.logdens(obs_ds))

#             self.create_summary(tf.summary.scalar, 'post_sigma', obs_ds[0])
            
    def predict(self, observable_mask, outputs):
        dim = self.dim
        data = self.data
        out = tf.reshape(outputs, [self.NUM_STEPS, dim[0], dim[1]])

        def step(prev, x):
            mask = x[0]
            prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
            params = x[2]

            d0 = params[:,:dim[0]]
            d1 = params[:,dim[0]:2*dim[0]]

            pp1 = prev_pred[:,:dim[0]]
            pp0 = prev_pred[:,dim[0]:2*dim[0]]

            new_pred = tf.matmul(pp0, d0)[0] + tf.matmul(pp1, d1)[0]+ params[:,-1] + pp1[0]
            new_pred = tf.concat([new_pred, pp1[0]], axis=0)
            return new_pred

        ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([2*dim[0]], dtype=floatX))
        return ar
    
    def create_likelihood(self, observable_mask, outputs):
        dim = self.dim
        obs_d = self.obs_d#Normal(dim=None, sigma=self.obs_ds, mu=0) #self.obs_d
        
        preds = self.predict(observable_mask, outputs)
        
        diffs = preds[:-1] - data[0,1:]
        print(diffs)
        diffs = diffs[:,:dim[0]]
        
        rsq_obs = tf.sqrt(tf.reduce_mean(tf.square(diffs), axis=0))/np.std(self.data[0,1:,:3] - self.data[0,:-1,:3], axis=0)
        rsq_obs = 1-tf.reduce_mean(rsq_obs)
        self.create_summary(tf.summary.scalar, 'rsquared_observed', rsq_obs)

        logl = obs_d.logdens(diffs, reduce=False)
        print(logl)
        logl *= tf.cast(self.observable_mask[:-1], floatX)
        print(logl)

        logl = tf.reduce_sum(logl)
        self.create_summary(tf.summary.scalar, 'loglikelihood', logl)
        self.priors.append(logl)

In [13]:
model = VARmodel(datas[0], name='AUS')

0.4 0.0
0.4 0.0
Tensor("AUS/sub_1:0", shape=(34, 6), dtype=float64)
Tensor("AUS/obs_d_posterior_2/sub_1:0", shape=(34,), dtype=float64)
Tensor("AUS/mul:0", shape=(34,), dtype=float64)


In [14]:
prior = tf.reduce_sum(model.priors)

logdensity = tf.reduce_sum(model.logdensities)

In [15]:
kl = logdensity - prior

In [16]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge([kls, tf.summary.scalar('prior', prior)] + [model.summaries])

In [17]:
main_op = tf.train.AdamOptimizer(0.0001).minimize(kl)

In [18]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [19]:
init.run()

In [20]:
# !rm -R /tmp/hier

In [21]:
writer = tf.summary.FileWriter('/tmp/hier/single_full_stochcov-ebayes-notrestrictive-correctrsq-correct')

In [22]:
writer.add_graph(tf.get_default_graph())

In [23]:
epoch = 0

In [ ]:
for epoch in range(epoch, 100000):
    for step in range(100):
        sess.run(main_op)
    s, _ = sess.run([summary, main_op])
    writer.add_summary(s, global_step=epoch)

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(1000):
    ss.append(global_post.eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,4], ss[:,1])
plt.show()